In [2]:
import keras
import keras.backend as K
from keras.callbacks import CSVLogger
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np

from tensorflow.keras.models import Model
from tensorflow.keras.layers import MaxPooling2D,Conv2D,Dense,BatchNormalization,concatenate,Input,Dropout,Maximum,Activation,Dense,Flatten,UpSampling2D,Conv2DTranspose
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.initializers as initializers
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import regularizers

from medpy import metric

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

2023-04-24 19:49:45.259746: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-24 19:49:45.294286: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-24 19:49:45.885837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
def unet2d():
    ############ Encoder #############
    input_ = Input(shape=(128,128,4),name='input')                                              

    block1_conv1 = Conv2D(64,3,padding='same',activation='relu',name='block1_conv1')(input_)
    block1_conv2 = Conv2D(64,3,padding='same',activation='relu',name='block1_conv2')(block1_conv1)
    block1_norm = BatchNormalization(name='block1_batch_norm')(block1_conv2)
    block1_pool = MaxPooling2D(name='block1_pool')(block1_norm)

    block2_conv1 = Conv2D(128,3,padding='same',activation='relu',name='block2_conv1')(block1_pool)
    block2_conv2 = Conv2D(128,3,padding='same',activation='relu',name='block2_conv2')(block2_conv1)
    block2_norm = BatchNormalization(name='block2_batch_norm')(block2_conv2)
    block2_pool = MaxPooling2D(name='block2_pool')(block2_norm)

    encoder_dropout_1 = Dropout(0.2,name='encoder_dropout_1')(block2_pool)

    block3_conv1 = Conv2D(256,3,padding='same',activation='relu',name='block3_conv1')(encoder_dropout_1)
    block3_conv2 = Conv2D(256,3,padding='same',activation='relu',name='block3_conv2')(block3_conv1)
    block3_norm = BatchNormalization(name='block3_batch_norm')(block3_conv2)
    block3_pool = MaxPooling2D(name='block3_pool')(block3_norm)

    block4_conv1 = Conv2D(512,3,padding='same',activation='relu',name='block4_conv1')(block3_pool)
    block4_conv2 = Conv2D(512,3,padding='same',activation='relu',name='block4_conv2')(block4_conv1)
    block4_norm = BatchNormalization(name='block4_batch_norm')(block4_conv2)
    block4_pool = MaxPooling2D(name='block4_pool')(block4_norm)
    ################### Encoder end ######################

    block5_conv1 = Conv2D(1024,3,padding='same',activation='relu',name='block5_conv1')(block4_pool)
    # encoder_dropout_2 = Dropout(0.2,name='encoder_dropout_2')(block5_conv1)

    ########### Decoder ################

    up_pool1 = Conv2DTranspose(1024,3,strides = (2, 2),padding='same',activation='relu',name='up_pool1')(block5_conv1)
    merged_block1 = concatenate([block4_norm,up_pool1],name='merged_block1')
    decod_block1_conv1 = Conv2D(512,3, padding = 'same', activation='relu',name='decod_block1_conv1')(merged_block1)

    up_pool2 = Conv2DTranspose(512,3,strides = (2, 2),padding='same',activation='relu',name='up_pool2')(decod_block1_conv1)
    merged_block2 = concatenate([block3_norm,up_pool2],name='merged_block2')
    decod_block2_conv1 = Conv2D(256,3,padding = 'same',activation='relu',name='decod_block2_conv1')(merged_block2)

    decoder_dropout_1 = Dropout(0.2,name='decoder_dropout_1')(decod_block2_conv1)

    up_pool3 = Conv2DTranspose(256,3,strides = (2, 2),padding='same',activation='relu',name='up_pool3')(decoder_dropout_1)
    merged_block3 = concatenate([block2_norm,up_pool3],name='merged_block3')
    decod_block3_conv1 = Conv2D(128,3,padding = 'same',activation='relu',name='decod_block3_conv1')(merged_block3)

    up_pool4 = Conv2DTranspose(128,3,strides = (2, 2),padding='same',activation='relu',name='up_pool4')(decod_block3_conv1)
    merged_block4 = concatenate([block1_norm,up_pool4],name='merged_block4')
    decod_block4_conv1 = Conv2D(64,3,padding = 'same',activation='relu',name='decod_block4_conv1')(merged_block4)
    ############ Decoder End ######################################

    # decoder_dropout_2 = Dropout(0.2,name='decoder_dropout_2')(decod_block4_conv1)

    pre_output = Conv2D(64,1,padding = 'same',activation='relu',name='pre_output')(decod_block4_conv1)

    output = Conv2D(4,1,padding='same',activation='softmax',name='output')(pre_output)

    model = Model(inputs = input_, outputs = output)
    # model.summary()
    return model

In [4]:
def dice_coef(y_true, y_pred, epsilon=1e-6):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection) / (K.sum(K.square(y_true),axis=-1) + K.sum(K.square(y_pred),axis=-1) + epsilon)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [5]:
# load_data
raw_data = np.load('data_fold_1.npy')
raw_gt = np.load('gt_fold_1.npy')

process_data = np.transpose(raw_data, (0, 2, 3, 4, 1))

# print(process_data.shape, raw_gt.shape)

process_data = process_data[:,30:120,30:222,30:222,:].reshape([-1,128,128,4])
process_gt = raw_gt[:,30:120,30:222,30:222].reshape([-1,128,128,1])
print(process_data.shape, process_gt.shape)


process_gt[np.where(process_gt==4)]=3
# unet2d()
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(process_data, process_gt, test_size=0.3, random_state=42)
# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25,random_state=42)

from keras.utils import to_categorical,normalize
Y_train = to_categorical(Y_train)
Y_val = to_categorical(Y_val)
X_train = (X_train-np.mean(X_train))/np.max(X_train)
# X_test = (X_test-np.mean(X_test))/np.max(X_test)
X_val = (X_val-np.mean(X_val))/np.max(X_val)

model = unet2d()

model.compile(optimizer=Adam(lr=1e-5),loss=dice_coef_loss,metrics=[dice_coef])
# model.load_weights('./Model Checkpoints/weights.hdf5')
checkpointer = callbacks.ModelCheckpoint(filepath = './Model Checkpoints/weights.hdf5', save_best_only=True)
training_log = callbacks.TensorBoard(log_dir='./Model_logs')
    


(19440, 128, 128, 4) (19440, 128, 128, 1)


2023-04-24 19:50:00.017017: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
print(X_train.shape, X_val.shape, Y_train.shape, Y_val.shape)

(13608, 128, 128, 4) (5832, 128, 128, 4) (13608, 128, 128, 4) (5832, 128, 128, 4)


In [7]:
history = model.fit(X_train,Y_train,validation_data=(X_val,Y_val),batch_size=48,epochs=10,callbacks=[training_log,checkpointer],shuffle=True)

Epoch 1/10
284/284 [==============================] - 897s 3s/step - loss: 0.0362 - dice_coef: 0.9638 - val_loss: 0.0321 - val_dice_coef: 0.9679
Epoch 2/10
284/284 [==============================] - 892s 3s/step - loss: 0.0321 - dice_coef: 0.9679 - val_loss: 0.0321 - val_dice_coef: 0.9679
Epoch 3/10
284/284 [==============================] - 894s 3s/step - loss: 0.0321 - dice_coef: 0.9679 - val_loss: 0.0321 - val_dice_coef: 0.9679
Epoch 4/10
284/284 [==============================] - 892s 3s/step - loss: 0.0321 - dice_coef: 0.9679 - val_loss: 0.0321 - val_dice_coef: 0.9679
Epoch 5/10
 33/284 [==>...........................] - ETA: 11:15 - loss: 0.0304 - dice_coef: 0.9696

KeyboardInterrupt: 

In [24]:
X_test, Y_test = np.load('data_fold_2.npy'), np.load('gt_fold_2.npy')


In [25]:
X_test = np.transpose(X_test, (0, 2, 3, 4, 1))
print(X_test.shape)
X_test = X_test[:,30:120,30:222,30:222,:].reshape([-1,128,128,4])

(96, 155, 240, 240, 4)


In [26]:
X_test.shape

(19440, 128, 128, 4)

In [27]:
X_test = (X_test-np.mean(X_test))/np.max(X_test)

In [31]:
model.load_weights('./Model Checkpoints/weights.hdf5')
Y_pre = np.argmax(model.predict(X_test),axis=-1)
print(np.unique(Y_pre))

608/608 [==============================] - 441s 725ms/step
[0]


In [32]:
print(Y_pre.shape)

(19440, 128, 128)


In [30]:
Y_test = Y_test[:,30:120,30:222,30:222].reshape([-1,128,128,1])
print(np.unique(Y_test))

[0 1 2 3]


In [33]:
from keras.utils import to_categorical
Y_test_encod = to_categorical(Y_test)
# print('HGG Test set no: 2 with Dice Coefficient:- %.4f'%float(list(model.evaluate(X_test2,Y_test_encod2,batch_size=32,verbose=2))[1]))
model.evaluate(X_test,Y_test_encod,verbose=0)


[0.029424650594592094, 0.9705751538276672]

In [35]:
Y_predict = model.predict(X_test)

608/608 [==============================] - 441s 725ms/step


In [36]:
Y_predict.shape, Y_test_encod.shape

((19440, 128, 128, 4), (19440, 128, 128, 4))

In [39]:
metric.hd95(Y_predict[:10], Y_test_encod[:10])

0.0

In [41]:
X_test, Y_test = np.load('data_fold_3.npy'), np.load('gt_fold_3.npy')
X_test = np.transpose(X_test, (0, 2, 3, 4, 1))
X_test = X_test[:,30:120,30:222,30:222,:].reshape([-1,128,128,4])
Y_test = Y_test[:,30:120,30:222,30:222].reshape([-1,128,128,1])
Y_test_encod = to_categorical(Y_test)

model.load_weights('./Model Checkpoints/weights.hdf5')
model.evaluate(X_test,Y_test_encod,verbose=0)

[0.03490065783262253, 0.9650992155075073]

In [42]:
X_test, Y_test = np.load('data_fold_4.npy'), np.load('gt_fold_4.npy')
X_test = np.transpose(X_test, (0, 2, 3, 4, 1))
X_test = X_test[:,30:120,30:222,30:222,:].reshape([-1,128,128,4])
Y_test = Y_test[:,30:120,30:222,30:222].reshape([-1,128,128,1])
Y_test_encod = to_categorical(Y_test)

model.load_weights('./Model Checkpoints/weights.hdf5')
model.evaluate(X_test,Y_test_encod,verbose=0)

[0.02290712483227253, 0.9770921468734741]

In [43]:
X_test, Y_test = np.load('data_fold_5.npy'), np.load('gt_fold_5.npy')
X_test = np.transpose(X_test, (0, 2, 3, 4, 1))
X_test = X_test[:,30:120,30:222,30:222,:].reshape([-1,128,128,4])
Y_test = Y_test[:,30:120,30:222,30:222].reshape([-1,128,128,1])
Y_test_encod = to_categorical(Y_test)

model.load_weights('./Model Checkpoints/weights.hdf5')
model.evaluate(X_test,Y_test_encod,verbose=0)

[0.03188618645071983, 0.9681133031845093]

In [47]:
metric.hd95(Y_predict[:5000], Y_test_encod[:5000])

0.0